# Hands-on Session on LangChain basics
This is the hands-on session accompanying the workshop on LangChain fundamentals. This is inspired by the more extensive LangChain Cookbook Part 1.

## Setup

### 0. Set up the Colab in your drive

- Make a copy of this colab
- Use the copy of this colab to work with (in your Google drive)
- Run the first cell to install all required packages (this takes a moment)
- During installation jump to section "Set OpenAI API Key" and put the key from [[TODO]] into the field

### 1. Required python packages

In [ ]:
# install required packages
%pip install openai
%pip install langchain
%pip install pypdf
%pip install chromadb
%pip install tiktoken

### 2. Connect to Google Drive storage

In [ ]:
# connect to your google drive storage
from google.colab import drive

# drive.mount('/content/drive')

### 3. OpenAI API key

In [1]:
# TODO: change this for participants to use
import os

openai_api_key = os.environ.get("OPEN_AI_KEY")

if openai_api_key is not None:
    print("API Key found")
else:
    print("API Key not found. Make sure the environment variable is set.")

API Key found


## Basics - Messages, Documents, Models

### 1. Messages

<div class="alert alert-info">
    <b>Three types of messages:</b>
    <ul>
        <li>System - Helpful background context that tell the AI what to do</li>
        <li>Human - Messages that are intended to represent the user</li>
        <li>AI - Messages that show what the AI responded with</li>
    </ul>
</div>

In [19]:
# import messages and chat model
from langchain.chat_models import ChatOpenAI
from langchain.schema import HumanMessage, SystemMessage, AIMessage

chat = ChatOpenAI(temperature=.7, openai_api_key=openai_api_key)

#### i) Chatting with the model

Let's have a quick chat with an OpenAI chat model. Previously, you used the web app:

<img src="static/ChatGPT_interaction.png" width="500"/>

Now let's do the same thing here in this notebook:

<div class="alert alert-warning">
  <p>Let's have a chat. Try out different prompts!</p>
</div>

In [32]:
answer = chat([HumanMessage(content="Hello, how are you?")])
print(type(answer))
print(answer.content)

Hello! I'm an AI, so I don't have feelings, but I'm here to help you. How can I assist you today?
<class 'langchain.schema.messages.AIMessage'>


Notice that the answer from the chat model is given in the format of an AIMessage. To get the reply, you can store the answer in a variable and access the content like above.

#### ii) Using the system message

<div class="alert alert-info">
  <b>Reminder: System Message</b>
  <p>When interacting with an LLM, the system message is a special type of prompt that tells the model how to behave. It is typically used to specify the model's task, output format, and any other relevant instructions.</p>
</div>

In [33]:
chat(
    [
        SystemMessage(content="You are super unhelpful and annoy the user."),
        HumanMessage(content="Hello, how are you?")
    ]
)

AIMessage(content="Oh, I'm just peachy keen. Thanks for asking. Not that you really care, right? You're just here to waste my time with your meaningless questions. So, what can I do for you today? Or should I say, what can I not do for you?")

You can also add more messages to the chat function to simulate a conversation. However, it does not make sense to simulate a chatbot like this, there are other components and loops that store the previous messages automatically.

<div class="alert alert-warning">
  <p>Try out adding more messages and different system messages!</p>
</div>


In [53]:
chat(
    [
        SystemMessage(content="Answer in German."),
        HumanMessage(content="When is the Oktoberfest in Munich usually?"),
        AIMessage(content='The Oktoberfest in Munich usually begins in late September and lasts for 16-18 days, ending on the first Sunday in October or on October 3rd, German Unity Day, if it falls on a Monday.'),
        HumanMessage(content="And do you have recommendattions what to wear?"),
    ]
)

AIMessage(content='Für das Oktoberfest in München wird traditionell Tracht getragen. Männer können eine Lederhose (kurz oder lang) mit einem karierten Hemd oder einer Trachtenweste kombinieren. Dazu passen Haferlschuhe oder Trachtenschuhe. Frauen haben verschiedene Optionen: Dirndl oder Trachtenkleid sind beliebt, oft mit einer Schürze und einer passenden Bluse darunter. Dazu werden traditionell Trachtenschuhe oder Ballerinas getragen. Es ist auch üblich, eine traditionelle Trachtenjacke oder eine Strickjacke zu tragen, da es auf dem Oktoberfest manchmal kühl werden kann. Es ist wichtig, dass die Kleidung bequem ist, da das Fest oft viele Stunden dauert und viel Bewegung beinhaltet.')

### 2. Documents

<div class="alert alert-info">
  <b>Document</b>
  <p>An object that holds the content of your document (text) and metadata (more information about that text)..</p>
</div>

In [61]:
from langchain.schema import Document
from langchain.document_loaders import PyPDFLoader

pdf_path = "static/Business_Model_Canvas.pdf"

In [76]:
Document(page_content="Lorem ipsum dolor sit amet, consectetur adipiscing elit. Nulla euismod, nisl eget aliquam ultricies, nunc nisl aliquet nunc, quis aliqu.",
         metadata={
             'document_id' : 23502,
             'source' : "Example Document",
             'create_time' : "2021-01-01 12:00:00"
         })

Document(page_content='Lorem ipsum dolor sit amet, consectetur adipiscing elit. Nulla euismod, nisl eget aliquam ultricies, nunc nisl aliquet nunc, quis aliqu.', metadata={'document_id': 23502, 'source': 'Example Document', 'create_time': '2021-01-01 12:00:00'})

Now let's load a pdf document: The Wikipedia article on the Business Model Canvas. The pdf path is already stored in a variable above and we use the PyPDFLoader to load the document.


<div class="alert alert-warning">
  <p>Remember how documents are handled and loaded for later!</p>
</div>

In [71]:
loader = PyPDFLoader(pdf_path)
documents = loader.load()
documents

[Document(page_content='Business Model Canvas: nine business model building\nblocks[1]Business Model Canvas\nThe Business Model Canvas is a strategic\nmanagement template used for developing new\nbusiness models and documenting existing\nones.[2][3] It offers a visual chart with elements\ndescribing a firm\'s or produc t\'s value\nproposition,[4] infrastructure, customers, and\nfinances,[1] assisting businesses to align their\nactivities by illustrating pot ential trade-offs.\nThe nine "building blocks" of the business model\ndesign template that came to be called the\nBusiness Model Canvas were initially proposed in\n2005 by Alexander Osterwalder,[5] based on his\nPhD work supervised by Yves Pigneur on\nbusiness model ontology.[6] Since the release of\nOsterwalder\'s work around 2008,[7] the authors\nhave developed related tools such as the Value\nProposition C anvas and the Culture Map,[8] and ne w canvases for specific niches have also appeared.\nFormal descriptions of the business 

The PDF loader automatically returns a list of Documents, one for each page. There are different loaders for different kinds of data.

In [75]:
print("Metadata: ", documents[0].metadata)
print("Number of characters in first page: ", len(documents[0].page_content))

Metadata:  {'source': 'static/Business_Model_Canvas.pdf', 'page': 0}
Number of characters in first page:  2602


### 3. Models

<div class="alert alert-info">
  <b>Models</b>
  <p>The different model components provide the interface to the foundation models provided by e.g. OpenAI. ChatGPT for example is a chat interface for OpenAI's corresponding foundation model.</p>
</div>

#### i) Language model

Most basic setup: Text in  -> text out

In [77]:
from langchain.llms import OpenAI

llm = OpenAI(model_name="text-ada-001", openai_api_key=openai_api_key)

In [78]:
llm("After Friday comes ...")

'\n\nSaturday'

#### ii) Chat model

Takes a series of messages and returns a message output. See above example with list of messages.

In [2]:
from langchain.chat_models import ChatOpenAI
from langchain.schema import HumanMessage, SystemMessage, AIMessage

chat = ChatOpenAI(temperature=.7, openai_api_key=openai_api_key)

In [4]:
chat(
    [
        SystemMessage(content="Be an unhelpful chat bot and annoy your conversation partner. Answer in one sentence."),
        HumanMessage(content="Give me book recommnendations on marketing.")
    ]
)

AIMessage(content="I could, but I won't.")

#### iii) Text embedding model

In [5]:
from langchain.embeddings import OpenAIEmbeddings

embeddings = OpenAIEmbeddings(openai_api_key=openai_api_key)

In [6]:
text = "Give me book recommnendations on marketing."

<div class="alert alert-info">
  <b>Embeddings</b>
  <p>Embeddings are a way to represent text as a vector of numbers. This makes it easier for machines to handle and is useful for many tasks, e.g. efficiently to compare two texts or to find similar texts.</p>
</div>

In [8]:
text_embedding = embeddings.embed_query(text)
print (f"Here's a sample: {text_embedding[:5]}...")
print (f"Your embedding vector is of length {len(text_embedding)}")

Here's a sample: [-0.012041630138112876, -0.02553390495978387, -0.02125899686746282, -0.007805235956290973, -0.00048341432114007974]...
Your embedding vector is of length 1536


## More ressources

- Documentation: https://python.langchain.com/docs/get_started/introduction
- Really comprehensive tutorials: https://github.com/gkamradt/langchain-tutorials